In [1]:
import os
import requests_html
import pandas as pd
import nasdaqdatalink as ndl
from dotenv import load_dotenv
import tkinter as tk
from tkinter import ttk, messagebox
import datetime
from yahoo_fin import stock_info as si


In [8]:
tickers_sp500 = si.tickers_sp500()

In [11]:
print(tickers_sp500)

['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '--', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---', '---',

In [12]:
import pandas as pd

def get_tickers_from_wikipedia():
    """Fetch a list of S&P 500 tickers from Wikipedia."""
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    df = tables[0]  # The first table usually contains the S&P 500 list
    tickers = df['Symbol'].tolist()
    return tickers

tickers = get_tickers_from_wikipedia()
print(tickers)


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', '

In [10]:
from yahoo_fin import stock_info as si
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_tickers():
    """Fetch a list of S&P 500 tickers."""
    tickers = si.tickers_sp500()
    print(f"Retrieved {len(tickers)} S&P 500 tickers.")
    return tickers

def fetch_trade_volume(ticker, start_date, end_date):
    """Fetch trade volume data for a single ticker."""
    if not ticker:
        print(f"Skipping empty ticker.")
        return ticker, 0
    
    try:
        print(f"Fetching data for ticker: {ticker}")
        df = si.get_data(ticker, start_date=start_date, end_date=end_date)
        if df.empty:
            print(f"No data returned for {ticker}.")
            return ticker, 0
        total_volume = df['volume'].sum()
        print(f"Fetched volume for {ticker}: {total_volume}")
        return ticker, total_volume
    except Exception as e:
        print(f"Error fetching volume for {ticker}: {e}")
        return ticker, 0

def fetch_volumes_parallel(tickers, start_date, end_date):
    """Fetch trade volume data for multiple tickers in parallel."""
    volumes = {}
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_trade_volume, ticker, start_date, end_date) for ticker in tickers]
        for future in as_completed(futures):
            ticker, volume = future.result()
            if volume > 0:
                volumes[ticker] = volume
    print(f"Fetched volumes for {len(volumes)} tickers.")
    return volumes

def get_top_tickers_by_volume(volumes, top_n=10):
    """Get the top N tickers by trade volume."""
    sorted_volumes = sorted(volumes.items(), key=lambda x: x[1], reverse=True)
    top_tickers = [ticker for ticker, volume in sorted_volumes[:top_n]]
    print(f"Top {top_n} tickers by trade volume:")
    return top_tickers

def print_top_tickers(start_date, end_date, top_n=10):
    """Fetch tickers, get their trade volumes, and print the top N tickers by trade volume."""
    # Get a list of tickers
    tickers = get_tickers()
    
    # Print the tickers being processed (for debugging)
    print(f"Processing {len(tickers)} tickers.")
    
    # Fetch trade volume for tickers in parallel
    volumes = fetch_volumes_parallel(tickers, start_date, end_date)
    
    # Get top tickers by volume
    top_tickers = get_top_tickers_by_volume(volumes, top_n)
    
    # Print the top tickers
    for ticker in top_tickers:
        print(ticker)

# Example usage
if __name__ == "__main__":
    # Define your date range
    start_date = '2023-01-01'
    end_date = datetime.now().strftime('%Y-%m-%d')  # Current date
    
    # Print the top 10 tickers by trade volume
    print_top_tickers(start_date, end_date)


Retrieved 503 S&P 500 tickers.
Processing 503 tickers.
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Fetching data for ticker: -
Error fetching volume for -: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Fetching data for ticker: --
Error fetching volume for -: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching volume for -: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching volume for -: {'chart': {'result': None, 'error': {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}}}
Error fetching volume for -: {'chart': {'result': None, 